In [65]:
import sys
sys.path.append('../')

import numpy as np
import inspect

import models.ac_stark as stark
import models.rydberg_calcs as ryd
import plotters.plot_ac_stark as pac

from scipy.constants import c as C_c, h as C_h, k as C_k, e as C_e
from scipy.constants import physical_constants, pi, epsilon_0, hbar

In [73]:
def E(P, w):
    # Intensity/beam (W/m^2)
    I = 2.0 * P / (np.pi * w**2)
    # Electric field (V/m)
    E = np.sqrt(2.0 * I / (epsilon_0 * C_c))
    return E

In [205]:
from arc import Cesium as Cs
cs = Cs()

def Omega(state1, state2, P, w, q):
    HFS = [np.shape(state1)[0] == 5, np.shape(state2)[0] == 5]
    
    n1, l1, j1 = (state1[0], state1[1], state1[2])
    n2, l2, j2 = (state2[0], state2[1], state2[2])
    rme = cs.getReducedMatrixElementJ(n1, l1, j1, n2, l2, j2)
    # print(rme)
    rme *= C_e * physical_constants["Bohr radius"][0]
    
    if HFS == [True, True]:
        # print("Both HFS")
        f1, mf1 = (state1[3], state1[4])
        f2, mf2 = (state2[3], state2[4])
        ame = (cs.getSphericalDipoleMatrixElement(f1, mf1, f2, mf2, q)
               * cs._reducedMatrixElementFJ(j1, f1, j2, f2))
        # print(ame)

    #NOTE: These two situations (where only one state considers HFS) should
    # only be used when we are seeing how much some HF state couples up to 
    # ANY of the J states. getSphericalMatrixElementHFStoFS automatically sets
    # the mI value of the J state so that we are coupled.
    elif HFS == [True, False]:
        # print("State 1 HFS")
        f1, mf1 = (state1[3], state1[4])
        mj2 = state2[3]
        ame = cs.getSphericalMatrixElementHFStoFS(j1, f1, mf1, j2, mj2, q)
        # print(ame)

    elif HFS == [False, True]:
        # print("State 2 HFS")
        mj1 = state1[3]
        f2, mf2 = (state2[3], state2[4])
        ame = cs.getSphericalMatrixElementHFStoFS(j2, f2, mf2, j1, mj1, q)
        # print(ame)
    
    elif HFS == [False, False]:
        # print("Neither HFS")
        mj1 = state1[3]
        mj2 = state2[3]
        ame = cs.getSphericalDipoleMatrixElement(j1, mj1, j2, mj2, q)
    
    Omega = E(P, w) * rme * ame / hbar
    
    return Omega

def Delta(state1, state2, wavelength):
    HFS = [np.shape(state1)[0] == 5, np.shape(state2)[0] == 5]
    
    n1, l1, j1 = (state1[0], state1[1], state1[2])
    n2, l2, j2 = (state2[0], state2[1], state2[2])
    
    E_1 = cs.getEnergy(n1, l1, j1)*C_e/C_h
    E_2 = cs.getEnergy(n2, l2, j2)*C_e/C_h

    if HFS[0]:
        f1 = state1[3]
        [A, B] = cs.getHFSCoefficients(n1, l1, j1)
        Ehfs = cs.getHFSEnergyShift(j1, f1, A, B)
        # print(Ehfs*1e-9 / (2.0 * np.pi))
        E_1 += Ehfs
    if HFS[1]: 
        f2 = state2[3]
        [A, B] = cs.getHFSCoefficients(n2, l2, j2)
        Ehfs = cs.getHFSEnergyShift(j2, f2, A, B)
        # print(Ehfs*1e-9 / (2.0 * np.pi))
        E_1 += Ehfs

    freq = C_c / wavelength
    print(freq)

    # returns in units of rad/s so energy is hbar times this.
    return 2*np.pi*(freq - np.abs(E_2 - E_1))
    

# def JandItoHFS

In [209]:
state_e = [7,1,3/2,5]
state_g = [6,0,1/2,4]

wavelength = C_c / (657932.608*1e9)

Delta(state_g, state_e, wavelength) * 1e-9 / (2*np.pi)

657932608000000.0


np.float64(-3.7155648515)

In [175]:
P_c= 100e-3
w_c = 25e-6
P_p = 1e-3
w_p = 25e-6

Delta = 3.3e9

state_r = [41,0,1/2,4,4]
state_e = [7,1,3/2,5,5]
state_g = [6,0,1/2,4,4]


OmC = Omega(state_e, state_r, P_c, w_c, -1) * 1e-6 / (2*np.pi)
OmP = Omega(state_g, state_e, P_p, w_p, 1) * 1e-6 / (2*np.pi)
OmR = OmC * OmP / (2*(Delta * 1e-6))

Pop_e_p = 0.5 * (OmP**2) / (2 * (Delta * 1e-6) ** 2)
Pop_e_c = 0.5 * (OmC**2) / (2 * (Delta * 1e-6) ** 2)
Gamma = 1.0 / cs.getStateLifetime(state_e[0], state_e[1], state_e[2])

print(OmP, OmC)
print(OmR)
print(Pop_e_p * Gamma * 1e-6, Pop_e_c * Gamma * 1e-6)

103.79033254715002 -83.97551587411672
-1.320582836120159
0.001905609071116621 0.0012474563713434266


In [151]:
P = 1
w = 25e-6
state2 = [41,0,1/2,4,4]
state1 = [7,1,3/2,5,5]

Omega(state1, state2, P, w, -1) * 1e-6 / (2*np.pi)

np.float64(-265.5538978498344)

In [159]:
P = 1e-3
w = 25e-6
state2 = [7,1,3/2,5,5]
state1 = [6,0,1/2,4,4]

Omega(state1, state2, P, w, 1) * 1e-6 / (2*np.pi)

np.float64(103.79033254715002)

In [140]:
P = 1
w = 25e-6
q = -1
state2 = [41,0,1/2,-1/2]
state1 = [7,1,3/2,5,5]
omega = Omega(state1, state2, P, w, q) * 1e-6 / (2*np.pi)
print(omega)

Omega_squared = 0
for i in range(-5, 6, 1):
    state1 = [7,1,3/2,5,i]
    Omega_squared += (Omega(state1, state2, P, w, q) * 1e-6 / (2*np.pi))**2

for i in range(-4, 5, 1):
    state1 = [7,1,3/2,4,i]
    Omega_squared += (Omega(state1, state2, P, w, q) * 1e-6 / (2*np.pi))**2

for i in range(-3, 4, 1):
    state1 = [7,1,3/2,3,i]
    Omega_squared += (Omega(state1, state2, P, w, q) * 1e-6 / (2*np.pi))**2

for i in range(-2, 3, 1):
    state1 = [7,1,3/2,2,i]
    Omega_squared += (Omega(state1, state2, P, w, q) * 1e-6 / (2*np.pi))**2

print(np.sqrt(Omega_squared))

-0.0
433.64769929284085
